1.Lets install and invoke packages.

In [1]:
!pip install flair
from tqdm.notebook import tqdm
from transformers import BertTokenizer, BertForTokenClassification
import requests,os
from flair.datasets import ColumnCorpus
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.data import Sentence
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
print(os.getenv("PYTORCH_CUDA_ALLOC_CONF"))

/home/sginne/anaconda3/envs/flair_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


expandable_segments:True


2-Loading tokenizers, models, etc

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=9)
train_url = 'https://raw.githubusercontent.com/lang-uk/flair-ner/main/fixed-split/train.iob'
test_url = 'https://raw.githubusercontent.com/lang-uk/flair-ner/main/fixed-split/test.iob'

def download_data(url, file_name):
    if not os.path.exists(file_name):
        response = requests.get(url)
        with open(file_name, 'w', encoding='utf-8') as f:
            f.write(response.text)
        print(f'Data downloaded: {file_name}')
    else:
        print('Data found:', file_name)
data_folder = './data/'
os.makedirs(data_folder, exist_ok=True)
download_data(train_url, os.path.join(data_folder, 'train.iob'))
download_data(test_url, os.path.join(data_folder, 'test.iob'))

distilbert_embeddings = TransformerWordEmbeddings('distilbert-base-multilingual-cased', fine_tune=True)


/home/sginne/anaconda3/envs/flair_env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Data found: ./data/train.iob
Data found: ./data/test.iob


Lets make corpus

In [3]:
columns = {0: 'text', 1: 'ner'}
corpus = ColumnCorpus(data_folder, columns,
                      train_file='train.iob',
                      test_file='test.iob')
print(corpus)
for i in range(15):
    print(corpus.train[i])
tag_types = corpus.make_label_dictionary(label_type='ner')
print(tag_types)




2024-09-07 23:53:29,910 Reading data from data
2024-09-07 23:53:29,912 Train: data/train.iob
2024-09-07 23:53:29,913 Dev: None
2024-09-07 23:53:29,913 Test: data/test.iob
2024-09-07 23:53:32,649 No dev split found. Using 10% (i.e. 876 samples) of the train split as dev data
Corpus: 7886 train + 876 dev + 4045 test sentences
Sentence[18]: "Зрозуміло , що український бізнес почав використовувати КСВ як інструмент формування своєї репутації буквально декілька років тому ."
Sentence[22]: "З одного боку , саме через це більшість проектів КСВ здійснюються епізодично та деколи виглядають , радше , як просто благодійність ."
Sentence[8]: "Винятком будуть хіба що представництва іноземних корпорацій ."
Sentence[20]: "З іншого боку , для українських компаній відкривається потужне « вікно можливостей » , щоб втілювати власні унікальні стратегії ."
Sentence[3]: "Філантропія як пріоритет"
Sentence[15]: "Це може бути як власна ініціатива , так і відповідь на запит від громади ."
Sentence[17]: "Це абс

8it [00:00, 6633.93it/s]
7886it [00:00, 32802.54it/s]

2024-09-07 23:53:32,898 Dictionary created for label 'ner' with 4 values: PERS (seen 2408 times), LOC (seen 1087 times), ORG (seen 453 times), MISC (seen 373 times)
Dictionary with 4 tags: PERS, LOC, ORG, MISC


Lets make tagger and trainer

In [4]:
tagger = SequenceTagger(hidden_size=128,
                        embeddings=distilbert_embeddings,
                        tag_dictionary=tag_types,
                        tag_type='ner',
                        use_crf=False)
trainer = ModelTrainer(tagger, corpus)


2024-09-07 23:53:32,906 SequenceTagger predicts: Dictionary with 17 tags: O, S-PERS, B-PERS, E-PERS, I-PERS, S-LOC, B-LOC, E-LOC, I-LOC, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC


Model name and and model

In [5]:
finetuned_model_name='dist-finetuned.05'


In [6]:
trainer.train(finetuned_model_name,
              learning_rate=0.01,
              mini_batch_size=1,
              max_epochs=48,
              #embeddings_storage_mode='cpu',
              #main_evaluation_metric='f1',
              monitor_test=True,
              use_amp=False,
              train_with_dev=True) 

2024-09-07 23:53:39,051 ----------------------------------------------------------------------------------------------------
2024-09-07 23:53:39,053 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(119548, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_featu

/home/sginne/anaconda3/envs/flair_env/lib/python3.8/site-packages/flair/trainers/trainer.py:499: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2024-09-07 23:54:35,008 epoch 1 - iter 876/8762 - loss 0.30177201 - time (sec): 55.94 - samples/sec: 288.02 - lr: 0.010000 - momentum: 0.000000
2024-09-07 23:55:26,159 epoch 1 - iter 1752/8762 - loss 0.25529109 - time (sec): 107.09 - samples/sec: 292.68 - lr: 0.010000 - momentum: 0.000000
2024-09-07 23:56:16,638 epoch 1 - iter 2628/8762 - loss 0.21538985 - time (sec): 157.57 - samples/sec: 295.24 - lr: 0.010000 - momentum: 0.000000
2024-09-07 23:57:07,196 epoch 1 - iter 3504/8762 - loss 0.19449142 - time (sec): 208.13 - samples/sec: 299.88 - lr: 0.010000 - momentum: 0.000000
2024-09-07 23:57:57,915 epoch 1 - iter 4380/8762 - loss 0.18189258 - time (sec): 258.85 - samples/sec: 303.79 - lr: 0.010000 - momentum: 0.000000
2024-09-07 23:58:48,780 epoch 1 - iter 5256/8762 - loss 0.17395712 - time (sec): 309.71 - samples/sec: 307.74 - lr: 0.010000 - momentum: 0.000000
2024-09-07 23:59:39,334 epoch 1 - iter 6132/8762 - loss 0.16982783 - time (sec): 360.26 - samples/sec: 309.24 - lr: 0.010000 -

100%|██████████| 64/64 [01:03<00:00,  1.01it/s]

2024-09-08 00:03:14,377 TEST : loss 0.1391058713197708 - f1-score (micro avg)  0.6948


...

100%|██████████| 64/64 [00:06<00:00, 10.05it/s]


2024-09-08 06:48:43,028 TEST : loss 0.27717503905296326 - f1-score (micro avg)  0.7119
2024-09-08 06:48:43,231  - 2 epochs without improvement
2024-09-08 06:48:48,573 ----------------------------------------------------------------------------------------------------
2024-09-08 06:48:48,605 Testing using last state of model ...


100%|██████████| 64/64 [00:06<00:00,  9.59it/s]


2024-09-08 06:48:56,290 
Results:
- F-score (micro) 0.7119
- F-score (macro) 0.4996
- Accuracy 0.6303

By class:
              precision    recall  f1-score   support

        PERS     0.9264    0.8248    0.8726      1678
         LOC     0.6012    0.7556    0.6696       401
         ORG     0.2873    0.6782    0.4036       261
        MISC     0.1231    0.0333    0.0525       240

   micro avg     0.6988    0.7256    0.7119      2580
   macro avg     0.4845    0.5730    0.4996      2580
weighted avg     0.7365    0.7256    0.7173      2580

2024-09-08 06:48:56,290 ----------------------------------------------------------------------------------------------------


{'test_score': 0.7119224187107815}

Lets evaluate it

In [7]:
result = tagger.evaluate(corpus.test, gold_label_type='ner')  # Evaluate on the test set
print(result.detailed_results)

100%|██████████| 127/127 [00:48<00:00,  2.61it/s]


Results:
- F-score (micro) 0.8194
- F-score (macro) 0.7011
- Accuracy 0.7506

By class:
              precision    recall  f1-score   support

        PERS     0.9090    0.8993    0.9041      1678
         LOC     0.7319    0.8304    0.7780       401
        MISC     0.5323    0.5500    0.5410       240
         ORG     0.6570    0.5211    0.5812       261

   micro avg     0.8210    0.8178    0.8194      2580
   macro avg     0.7075    0.7002    0.7011      2580
weighted avg     0.8210    0.8178    0.8181      2580



Lets test with own data

In [8]:
finetuned_model = SequenceTagger.load('/home/sginne/src/master_thesis/colab/'+finetuned_model_name+'/final-model.pt')
def print_tagged_sentences(sentences, model=None):
    # Load model only if not already loaded
    
    for text in sentences:
        sentence = Sentence(text)  # Create Sentence object
        model.predict(sentence)    # Predict tags
        #print (sentence.to_tagged_string())
        tagged_sentence = []
        spans=sentence.get_spans('ner')
    
        #for span in spans:
        #    print ("Span:",span.get_label('ner').value)
        #    print ("Span:",span.get_label('ner').score)
        i=0
        for token in sentence:
            if i!=len(spans):
                if (token==spans[i][0]):
                    current_tag_token=spans[i]
                    current_tag_value=current_tag_token.get_label('ner').value
                    current_tag_score=current_tag_token.get_label('ner').score
                    tagged_sentence.append(f'[{token.text}|({current_tag_value}={current_tag_score*100:.1f}%)]')
                    i=i+1
                else:
                    tagged_sentence.append(token.text)
            else:
                tagged_sentence.append(token.text)

        print ( ' '.join(tagged_sentence))

sentences = [
    'Привіт , мене звати Тімо і я живу в Турку , навчаюсь в Универсітеті Турку',
    'Тімо Юнолайнен живе в місті Турку й навчається в Університеті міста Турку.',
    'Місто Турку розташоване на південно-західному узбережжі Фінляндії.',
    'В Турку є поліція, й вона вельми корумпована',
    'У Турку проходить щорічний фестиваль музики та мистецтв.',
    'Тімо Юнолайнен, професор в Університеті міста Турку, співпрацює з організацією "Турку Інститут" для розробки нових проектів у Турку.'
]

print_tagged_sentences(sentences, finetuned_model)

2024-09-08 06:49:54,401 SequenceTagger predicts: Dictionary with 17 tags: O, S-PERS, B-PERS, E-PERS, I-PERS, S-LOC, B-LOC, E-LOC, I-LOC, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC
Привіт , мене звати [Тімо|(PERS=100.0%)] і я живу в [Турку|(LOC=100.0%)] , навчаюсь в [Универсітеті|(ORG=99.8%)] Турку
Тімо [Юнолайнен|(PERS=100.0%)] живе в місті [Турку|(LOC=100.0%)] й навчається в [Університеті|(ORG=89.9%)] міста Турку .
Місто [Турку|(LOC=100.0%)] розташоване на південно-західному узбережжі [Фінляндії|(LOC=100.0%)] .
В [Турку|(LOC=100.0%)] є поліція , й вона вельми корумпована
У [Турку|(LOC=100.0%)] проходить щорічний фестиваль музики та мистецтв .
Тімо [Юнолайнен|(PERS=74.0%)] , професор в [Університеті|(ORG=99.8%)] міста Турку , співпрацює з організацією " [Турку|(ORG=100.0%)] Інститут " для розробки нових проектів у [Турку|(LOC=100.0%)] .
